In [25]:
import numpy as np
import cvxpy as cp
from tabulate import tabulate

BitString = list[int]

# Set the possible values of each bit: {0, 1}
a_range = b_range = x_range = y_range = range(2)

def generate_bitstrings(n: int) -> list[BitString]:
    """
    Function that generates all possible bitstrings of length n.
    """
    if n == 0:
        return [[]]
    else:
        previous_bitstrings = generate_bitstrings(n - 1)
        current_bitstrings = []
        for bitstring in previous_bitstrings:
            current_bitstrings.append(bitstring + [0])
            current_bitstrings.append(bitstring + [1])
        return current_bitstrings

Here, the bitstring lengths that Alice and Bob receive can be set. $x=x\_range^n$ is for Alice, $y=y\_range^m$ is for Bob.

In [26]:
# These variables may be changed
n = 2
m = 2

# These variables shouldn't be changed
# Generates all possible bitstrings for x and y
x_values = generate_bitstrings(n)
y_values = generate_bitstrings(m)

$q(x,y)$ is the distribution function, which returns the probability of Alice receiving $x$ and Bob receiving $y$. For a uniform distribution, `q_uniform` can be called.

In [27]:
def q_uniform():
    x_possibilities = len(x_values)
    y_possibilities = len(y_values)

    return 1 / (x_possibilities * y_possibilities)

# This function may be changed
def q(x: BitString, y: BitString):
    return q_uniform()

Alice and Bob win the game when $a \oplus b = f(x,y)$.
$f(x,y)$ can be set here.

In [28]:
# This function may be changed
def f(x: BitString, y: BitString):
    return (x[0] * x[1]) * y[0]

We rewrite the calculation of the entangled bias:

$$\varepsilon^*(G) = \sum_{xy} q(x,y)(-1)^{f(x,y)}\langle \Psi | A_x \otimes B_y | \Psi \rangle$$
$$\varepsilon^*(G) = \langle D, M \rangle$$

where
$$D(x,y)=q(x,y)(-1)^{f(x,y)$$
and
$$M(x,y)=\langle \Psi | A_x \otimes B_y | \Psi \rangle$$

We will create the matrix $D$ here. $M$ is the matrix we want to optimize.

In [29]:
# This shouldn't be changed
def D_constructor(x: BitString, y: BitString):
    return q(x, y) * ((-1) ** f(x, y))

D = np.matrix([[D_constructor(x, y) for y in y_values] for x in x_values])
print(D)

[[ 0.0625  0.0625  0.0625  0.0625]
 [ 0.0625  0.0625  0.0625  0.0625]
 [ 0.0625  0.0625  0.0625  0.0625]
 [ 0.0625  0.0625 -0.0625 -0.0625]]


Here we start defining the semidefinite program. We start by creating the variables that need to be optimized.

From Tsirelson's theorem, remember

$$Z=\begin{pmatrix}
            R & M \\
            M^* & S
        \end{pmatrix}
        \geq 0$$
where the diagonal entries of $R$ and $S$ are 1.

In [30]:
# Define the variables of the optimization problem
M = cp.Variable(D.shape)
R = cp.Variable(D.shape)
S = cp.Variable(D.shape)

# Create block matrix Z
Z = cp.bmat([[R, M], [cp.conj(M), S]])

Now, we add the constraints to the semidefinite program.

In [31]:
# Z is semidefinite
constraints = [Z >> 0]

# All diagonal entries of Z need to be 1.
constraints += [cp.diag(Z) == np.ones(Z.shape[0])]

Finally, we solve the optimization problem

$$\varepsilon^*(G) = \max \langle D, M \rangle$$
under the constraints defined above. Note that

$$\max \langle D, M \rangle = \max \mathrm{Tr}(D^*M)$$

In [32]:
problem = cp.Problem(cp.Maximize(cp.trace(cp.conj(D) @ M)),
                     constraints)
problem.solve()

# Print results.
print("The optimal bias is", problem.value)
print("The optimal winning probability is", 0.5 + problem.value / 2)

The optimal bias is inf
The optimal winning probability is inf


We can also print the (now optimized) matrix $Z$.

In [33]:
table = tabulate(Z.value, tablefmt="simple_grid")
print(table)